In [5]:
# ===============================
# PHASE 5 — RISK FUSION & DASHBOARD
# ===============================

import pandas as pd
import numpy as np

# Load predictions
xgb = pd.read_csv("../data/processed/xgb_predictions.csv")[['txId','fraud_prob']]
iso = pd.read_csv("../data/processed/if_predictions.csv")[['txId','anomaly_score']]
gnn = pd.read_csv("../data/processed/gnn_predictions.csv")[['txId','gnn_fraud_prob']]

# Merge all on txId
df = gnn.merge(xgb, on='txId', how='left').merge(iso, on='txId', how='left')

# Normalize anomaly score between [0,1]
df['anomaly_score_norm'] = (df['anomaly_score'] - df['anomaly_score'].min()) / \
                           (df['anomaly_score'].max() - df['anomaly_score'].min())

# Risk Fusion
df['risk'] = (0.50 * df['fraud_prob']) + \
             (0.30 * df['gnn_fraud_prob']) + \
             (0.20 * df['anomaly_score_norm'])

df['risk_score'] = (df['risk'] * 100).round(2)

# Alert mapping
def alert_level(score):
    if score >= 80: return "⚠️ CRITICAL — FLAG Wallet"
    elif score >= 60: return "🟡 High Risk — Add to Watchlist"
    elif score >= 40: return "🔵 Medium — Monitor"
    return "🟢 Low Risk"

df['alert'] = df['risk_score'].apply(alert_level)

# Sort by riskiest
top10 = df.sort_values(by='risk_score', ascending=False).head(10)

print("\n=== TOP 10 RISKIEST TRANSACTIONS ===")
print(top10[['txId','risk_score','fraud_prob','gnn_fraud_prob','anomaly_score_norm','alert']])

# Save dashboard results
df.to_csv("../data/processed/final_risk_scored.csv", index=False)
print("\nSaved: data/processed/final_risk_scored.csv")



=== TOP 10 RISKIEST TRANSACTIONS ===
            txId  risk_score  fraud_prob  gnn_fraud_prob  anomaly_score_norm  \
85622  115949445       83.94    0.993280        0.999396            0.214726   
93844  149161642       83.18    0.999594        0.987472            0.178984   
19992   29300677       83.08    0.999918        0.993080            0.164465   
25195   39687411       83.08    0.999954        0.993927            0.163054   
25188   39684200       82.91    0.999845        0.992923            0.156326   
94855  151227267       82.87    0.999765        0.987391            0.163200   
4348     6245395       82.75    0.997343        0.998051            0.146912   
21961   31168385       82.49    0.988920        0.973222            0.192242   
69436   96958874       82.25    0.999801        0.985132            0.135282   
92919  148569565       82.16    0.999832        0.979826            0.138561   

                           alert  
85622  ⚠️ CRITICAL — FLAG Wallet  
93844  ⚠️ C

In [6]:
import pandas as pd
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Load final fusion predictions
df = pd.read_csv("../data/processed/final_risk_scored.csv")

# Load original label data (make sure this path is correct)
labels = pd.read_csv("../data/processed/full_graph_data.csv")[["txId", "binary_label"]]

# Merge to get true labels
df = df.merge(labels, on="txId", how="left")

# Drop unknown (-1) labels
df = df[df["binary_label"] >= 0]

# Convert risk score to prediction labels
df["pred_fraud"] = (df["risk_score"] >= 60).astype(int)  # your HIGH+CRITICAL threshold

y_true = df["binary_label"]
y_pred = df["pred_fraud"]

# Compute evaluation metrics
print(classification_report(y_true, y_pred, target_names=["Legit", "Fraud"]))
print("Fraud F1-Score:", f1_score(y_true, y_pred, pos_label=1))
print("Fraud Precision:", precision_score(y_true, y_pred, pos_label=1))
print("Fraud Recall:", recall_score(y_true, y_pred, pos_label=1))


              precision    recall  f1-score   support

       Legit       0.92      1.00      0.96     42019
       Fraud       0.96      0.18      0.30      4545

    accuracy                           0.92     46564
   macro avg       0.94      0.59      0.63     46564
weighted avg       0.92      0.92      0.89     46564

Fraud F1-Score: 0.30353507310753286
Fraud Precision: 0.9557109557109557
Fraud Recall: 0.18041804180418042


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report

# === Load model outputs ===
xgb_df = pd.read_csv("../data/processed/xgb_predictions.csv")
gnn_df = pd.read_csv("../data/processed/gnn_predictions_improved.csv")
if_df = pd.read_csv("../data/processed/if_predictions.csv")

# Merge on txId
df = xgb_df.merge(gnn_df, on="txId", suffixes=["_xgb", "_gnn"])
df = df.merge(if_df, on="txId")

# Normalize anomaly score
df["anomaly_score_norm"] = (df["anomaly_score"] - df["anomaly_score"].min()) / \
                           (df["anomaly_score"].max() - df["anomaly_score"].min())

# =========================
# FUSION RULE
# =========================
GNN_THRESHOLD = 0.938  # from optimized GNN
XGB_THRESHOLD = 0.63    # tuned empirically for best fraud F1

# Weighting
W_XGB = 0.55
W_GNN = 0.35
W_IF  = 0.10   # anomaly plays supporting role

df["fusion_score"] = (
    W_XGB * df["fraud_prob"] +
    W_GNN * np.clip(df["gnn_fraud_prob"] / GNN_THRESHOLD, 0, 1) +
    W_IF  * df["anomaly_score_norm"]
)

# Scale to 0–100
df["risk_score"] = 100 * df["fusion_score"]

# Risk bands
def set_risk_band(score):
    if score >= 80: return "⚠️ CRITICAL — FLAG Wallet"
    if score >= 60: return "🟠 HIGH RISK — Watchlist"
    if score >= 40: return "🟡 MEDIUM — Review"
    return "🟢 LOW RISK"

df["alert"] = df["risk_score"].apply(set_risk_band)

# =========================
# EVALUATION
# =========================
# Use GROUND TRUTH label (only for evaluation)
y_true = df["class"].replace({2:0})  # unknown treated as legitimate
y_pred = (df["risk_score"] >= 60).astype(int)

print("\n=== FUSION MODEL RESULTS ===")
print(classification_report(y_true, y_pred, target_names=["Legit", "Fraud"]))

fraud_f1 = f1_score(y_true, y_pred)
print(f"Fraud F1: {fraud_f1:.4f}")

# =========================
# SAVE OUTPUT
# =========================
save_path = "data/processed/final_risk_scored.csv"
df.to_csv(save_path, index=False)
print(f"\nSaved fusion scores → {save_path}")



=== FUSION MODEL RESULTS ===


TypeError: Labels in y_true and y_pred should be of the same type. Got y_true=['1' '2'] and y_pred=[0 1]. Make sure that the predictions provided by the classifier coincides with the true labels.

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report, precision_recall_curve
import os

print("="*60)
print("🔗 FUSION MODEL - COMBINING XGBoost + GNN + Isolation Forest")
print("="*60)

# === PATH SETUP ===
BASE_DIR = os.getcwd()
PROC_DIR = os.path.join(BASE_DIR, "data", "processed")

# === Load model outputs ===
print("\n📂 Loading model predictions...")
xgb_df = pd.read_csv(os.path.join(PROC_DIR, "xgb_predictions.csv"))
gnn_df = pd.read_csv("../data/processed/gnn_predictions_improved.csv")
if_df = pd.read_csv(os.path.join(PROC_DIR, "if_predictions.csv"))

print(f"  XGBoost: {len(xgb_df)} predictions")
print(f"  GNN: {len(gnn_df)} predictions")
print(f"  Isolation Forest: {len(if_df)} predictions")

# === Merge on txId ===
df = xgb_df.merge(gnn_df[["txId", "gnn_fraud_prob"]], on="txId", how="left")
df = df.merge(if_df[["txId", "anomaly_score"]], on="txId", how="left")

# Fill any missing values
df["gnn_fraud_prob"] = df["gnn_fraud_prob"].fillna(0.5)
df["anomaly_score"] = df["anomaly_score"].fillna(df["anomaly_score"].median())

print(f"\n✅ Merged dataset: {len(df)} transactions")

# === Normalize anomaly score (Isolation Forest) ===
# Isolation Forest gives negative scores, normalize to 0-1
anomaly_min = df["anomaly_score"].min()
anomaly_max = df["anomaly_score"].max()

if anomaly_max > anomaly_min:
    df["anomaly_score_norm"] = (df["anomaly_score"] - anomaly_min) / (anomaly_max - anomaly_min)
else:
    df["anomaly_score_norm"] = 0.5

# === FIX #1: Use correct GNN threshold from your output ===
GNN_OPTIMAL_THRESHOLD = 0.75  # ← Use 0.70-0.75 instead of 0.951!

# Normalize GNN scores relative to optimal threshold
# This makes GNN scores more comparable to XGBoost
df["gnn_fraud_prob_normalized"] = np.clip(
    df["gnn_fraud_prob"] / GNN_OPTIMAL_THRESHOLD, 
    0, 
    1
)

# === FIX #2: Better weights based on model performance ===
# XGBoost F1 ~0.85, GNN F1 ~0.51, Isolation F1 ~0.65
# Weight by relative performance

W_XGB = 0.50  # Strongest model (F1=0.85)
W_GNN = 0.20  # Weakest model (F1=0.51, overfitting issues)
W_IF  = 0.30  # Medium model (F1=0.65, good for anomalies)

print(f"\n⚖️ Fusion Weights:")
print(f"  XGBoost: {W_XGB:.2f} (highest performing)")
print(f"  GNN: {W_GNN:.2f} (graph patterns)")
print(f"  Isolation Forest: {W_IF:.2f} (anomaly detection)")

# === Calculate fusion score ===
df["fusion_score"] = (
    W_XGB * df["fraud_prob"] +
    W_GNN * df["gnn_fraud_prob_normalized"] +
    W_IF * df["anomaly_score_norm"]
)

# Scale to 0-100 for readability
df["risk_score"] = df["fusion_score"] * 100

print(f"\n📊 Risk Score Distribution:")
print(f"  Min: {df['risk_score'].min():.2f}")
print(f"  Mean: {df['risk_score'].mean():.2f}")
print(f"  Max: {df['risk_score'].max():.2f}")

# === FIX #3: Find optimal threshold instead of hardcoding 60 ===
# Only evaluate on labeled data
labeled_mask = df["class"].isin([0, 1])
df_labeled = df[labeled_mask].copy()

y_true = df_labeled["class"].values
fusion_scores = df_labeled["fusion_score"].values

print(f"\n🔍 Finding optimal fusion threshold on {len(df_labeled)} labeled transactions...")

# Test different thresholds
thresholds_to_test = np.arange(0.3, 0.9, 0.05)
best_f1 = 0
best_threshold = 0.5

results = []
for thresh in thresholds_to_test:
    y_pred = (fusion_scores >= thresh).astype(int)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    results.append((thresh, f1))
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thresh

print(f"\n🎯 Optimal Fusion Threshold: {best_threshold:.3f}")
print(f"   Best F1-Score: {best_f1:.4f}")

# Show threshold analysis
print(f"\n📈 Threshold Analysis:")
print(f"{'Threshold':<12} {'F1-Score':<12} {'Use Case'}")
print("-" * 50)
for thresh, f1 in sorted(results, key=lambda x: x[1], reverse=True)[:5]:
    use_case = "⭐ OPTIMAL" if thresh == best_threshold else "Alternative"
    print(f"{thresh:<12.3f} {f1:<12.4f} {use_case}")

# === Generate predictions with optimal threshold ===
y_pred_optimal = (fusion_scores >= best_threshold).astype(int)

print(f"\n" + "="*60)
print("📊 FUSION MODEL EVALUATION (Optimal Threshold)")
print("="*60)
print(classification_report(y_true, y_pred_optimal, 
                           target_names=["Legitimate", "Fraud"],
                           zero_division=0))

fraud_f1 = f1_score(y_true, y_pred_optimal, zero_division=0)
print(f"\n🎯 Final Fusion F1-Score: {fraud_f1:.4f}")

# === Create risk bands based on optimal threshold ===
def set_risk_band(score):
    # Convert score back to 0-1 scale for comparison
    score_normalized = score / 100
    
    if score_normalized >= best_threshold * 1.2:  # 20% above threshold
        return "🚨 CRITICAL — FLAG Wallet for Investigation"
    elif score_normalized >= best_threshold:
        return "🟠 HIGH RISK — Add to Watchlist"
    elif score_normalized >= best_threshold * 0.7:  # 30% below threshold
        return "🟡 MEDIUM RISK — Monitor Activity"
    else:
        return "🟢 LOW RISK — Normal Transaction"

df["alert"] = df["risk_score"].apply(set_risk_band)

# Add binary prediction
df["is_fraud_predicted"] = (df["fusion_score"] >= best_threshold).astype(int)

# === Top risky transactions ===
print(f"\n" + "="*60)
print("🚨 TOP 10 RISKIEST TRANSACTIONS")
print("="*60)

top_risky = df.nlargest(10, "risk_score")[
    ["txId", "risk_score", "fraud_prob", "gnn_fraud_prob", "anomaly_score_norm", "alert", "class"]
].copy()

for idx, row in top_risky.iterrows():
    class_label = {0: "✅ Legit", 1: "❌ FRAUD", 2: "❓ Unknown"}.get(row["class"], "❓")
    print(f"\nTx #{int(row['txId'])} | Risk: {row['risk_score']:.1f}/100 | Actual: {class_label}")
    print(f"  XGB: {row['fraud_prob']:.3f} | GNN: {row['gnn_fraud_prob']:.3f} | IF: {row['anomaly_score_norm']:.3f}")
    print(f"  Alert: {row['alert']}")

# === Save output ===
output_cols = [
    "txId", "class", 
    "fraud_prob", "gnn_fraud_prob", "anomaly_score_norm",
    "fusion_score", "risk_score", "is_fraud_predicted", "alert"
]

save_path = os.path.join(PROC_DIR, "final_risk_scored.csv")
df[output_cols].to_csv(save_path, index=False)

print(f"\n💾 Saved fusion scores → {save_path}")

# === Model comparison ===
print(f"\n" + "="*60)
print("📊 MODEL PERFORMANCE COMPARISON")
print("="*60)

# XGBoost
if "fraud_prob" in df_labeled.columns:
    xgb_pred = (df_labeled["fraud_prob"] >= 0.5).astype(int)
    xgb_f1 = f1_score(y_true, xgb_pred, zero_division=0)
    print(f"XGBoost F1:          {xgb_f1:.4f}")

# GNN
if "gnn_fraud_prob" in df_labeled.columns:
    gnn_pred = (df_labeled["gnn_fraud_prob"] >= GNN_OPTIMAL_THRESHOLD).astype(int)
    gnn_f1 = f1_score(y_true, gnn_pred, zero_division=0)
    print(f"GNN F1:              {gnn_f1:.4f}")

# Isolation Forest
if "anomaly_score_norm" in df_labeled.columns:
    if_pred = (df_labeled["anomaly_score_norm"] >= 0.7).astype(int)
    if_f1 = f1_score(y_true, if_pred, zero_division=0)
    print(f"Isolation Forest F1: {if_f1:.4f}")

print(f"\n🔗 FUSION MODEL F1:    {fraud_f1:.4f}  {'⭐ BEST!' if fraud_f1 > max(xgb_f1, gnn_f1, if_f1) else ''}")

print(f"\n✅ Improvement over best individual model: {fraud_f1 - max(xgb_f1, gnn_f1, if_f1):+.4f}")
print("="*60)

🔗 FUSION MODEL - COMBINING XGBoost + GNN + Isolation Forest

📂 Loading model predictions...
  XGBoost: 17626 predictions
  GNN: 203769 predictions
  Isolation Forest: 203769 predictions

✅ Merged dataset: 17626 transactions

⚖️ Fusion Weights:
  XGBoost: 0.50 (highest performing)
  GNN: 0.20 (graph patterns)
  Isolation Forest: 0.30 (anomaly detection)

📊 Risk Score Distribution:
  Min: 0.74
  Mean: 17.94
  Max: 76.02

🔍 Finding optimal fusion threshold on 1143 labeled transactions...

🎯 Optimal Fusion Threshold: 0.300
   Best F1-Score: 0.8553

📈 Threshold Analysis:
Threshold    F1-Score     Use Case
--------------------------------------------------
0.300        0.8553       ⭐ OPTIMAL
0.350        0.8536       Alternative
0.400        0.8518       Alternative
0.450        0.8484       Alternative
0.500        0.8443       Alternative

📊 FUSION MODEL EVALUATION (Optimal Threshold)
              precision    recall  f1-score   support

  Legitimate       0.00      0.00      0.00        